In [1]:
import requests
import json
import pprint
import pandas as pd
import pycurl
from itertools import chain
pp = pprint.PrettyPrinter()

In [2]:
# This is an example of one pull from the balldontlie.io API for a player searched by name
# You can see that lebron james was added as a search term at the end, and we can replicate this with other players
# The goal being to isolate the id's to use later for performance statistics
# Next we will be creating a for loop to get all of the players from our salaries list

In [3]:
# As you can see in this example, any capitalization does not matter for the player search
james = requests.get("https://www.balldontlie.io/api/v1/players/?search=LEbron jaMes")
james.json()

{'data': [{'id': 237,
   'first_name': 'LeBron',
   'height_feet': 6,
   'height_inches': 8,
   'last_name': 'James',
   'position': 'F',
   'team': {'id': 14,
    'abbreviation': 'LAL',
    'city': 'Los Angeles',
    'conference': 'West',
    'division': 'Pacific',
    'full_name': 'Los Angeles Lakers',
    'name': 'Lakers'},
   'weight_pounds': 250}],
 'meta': {'total_pages': 1,
  'current_page': 1,
  'next_page': None,
  'per_page': 25,
  'total_count': 1}}

In [4]:
# We take the file from BDLio and convert it to a dictionary so we car parse the ID
james_dict = json.loads(james.text)
print(james_dict)
type(james_dict)

{'data': [{'id': 237, 'first_name': 'LeBron', 'height_feet': 6, 'height_inches': 8, 'last_name': 'James', 'position': 'F', 'team': {'id': 14, 'abbreviation': 'LAL', 'city': 'Los Angeles', 'conference': 'West', 'division': 'Pacific', 'full_name': 'Los Angeles Lakers', 'name': 'Lakers'}, 'weight_pounds': 250}], 'meta': {'total_pages': 1, 'current_page': 1, 'next_page': None, 'per_page': 25, 'total_count': 1}}


dict

In [5]:
james_id = james_dict['data'][0]["id"]

In [6]:
%store -r player_names_list
player_names_list

['Stephen Curry',
 'John Wall',
 'Russell Westbrook',
 'James Harden',
 'Damian Lillard',
 'LeBron James',
 'Kevin Durant',
 'Giannis Antetokounmpo',
 'Paul George',
 'Kawhi Leonard',
 'Klay Thompson',
 'Jimmy Butler',
 'Tobias Harris',
 'Khris Middleton',
 'Anthony Davis',
 'Rudy Gobert',
 'Kyrie Irving',
 'Bradley Beal',
 'Kristaps Porzingis',
 'Devin Booker']

In [7]:
url = "https://www.balldontlie.io/api/v1/players/?search="
player_id_list = []

for name in player_names_list:
    player_url = url + f"{name}"
    player_get = requests.get(player_url)
    player_get.json()
    player_dict = json.loads(player_get.text)
    player_id_list.append(player_dict['data'][0]["id"])
print(player_id_list)


[115, 1014, 472, 192, 278, 237, 140, 15, 172, 274, 443, 79, 200, 315, 117, 176, 228, 37, 378, 57]


In [8]:
averages_url = "https://www.balldontlie.io/api/v1/season_averages?player_ids[]="
averages_list = []
for ids in player_id_list:
    averages_new = averages_url + f"{str(ids)}"
    averages_get = requests.get(averages_new)
    averages_get.json()
    averages_dict = json.loads(averages_get.text)
    print(averages_dict)
    if ids == 57:
        break
  

{'data': [{'games_played': 12, 'player_id': 115, 'season': 2021, 'min': '31:13', 'fgm': 8.0, 'fga': 18.33, 'fg3m': 4.75, 'fg3a': 12.0, 'ftm': 4.5, 'fta': 4.75, 'oreb': 0.75, 'dreb': 5.33, 'reb': 6.08, 'ast': 6.33, 'stl': 1.67, 'blk': 0.5, 'turnover': 3.0, 'pf': 1.33, 'pts': 25.25, 'fg_pct': 0.436, 'fg3_pct': 0.396, 'ft_pct': 0.947}]}
{'data': []}
{'data': [{'games_played': 12, 'player_id': 472, 'season': 2021, 'min': '34:12', 'fgm': 6.92, 'fga': 17.08, 'fg3m': 1.0, 'fg3a': 3.83, 'ftm': 2.83, 'fta': 4.42, 'oreb': 1.58, 'dreb': 7.0, 'reb': 8.58, 'ast': 8.17, 'stl': 1.5, 'blk': 0.17, 'turnover': 4.92, 'pf': 3.08, 'pts': 17.67, 'fg_pct': 0.405, 'fg3_pct': 0.261, 'ft_pct': 0.642}]}
{'data': [{'games_played': 12, 'player_id': 192, 'season': 2021, 'min': '33:53', 'fgm': 5.58, 'fga': 13.67, 'fg3m': 2.92, 'fg3a': 7.42, 'ftm': 4.08, 'fta': 4.67, 'oreb': 1.17, 'dreb': 6.83, 'reb': 8.0, 'ast': 8.75, 'stl': 1.08, 'blk': 0.83, 'turnover': 4.67, 'pf': 2.33, 'pts': 18.17, 'fg_pct': 0.409, 'fg3_pct': 0

In [10]:
averages_url = "https://www.balldontlie.io/api/v1/season_averages?player_ids[]="
averages_list = []
for ids in player_id_list:
    averages_new = averages_url + f"{str(ids)}"
    averages_get = requests.get(averages_new)
    averages_get.json()
    averages_dict = json.loads(averages_get.text)
    averages_list.append(averages_dict['data'])
    if ids == 57:
        break
averages_list

[[{'games_played': 12,
   'player_id': 115,
   'season': 2021,
   'min': '31:13',
   'fgm': 8.0,
   'fga': 18.33,
   'fg3m': 4.75,
   'fg3a': 12.0,
   'ftm': 4.5,
   'fta': 4.75,
   'oreb': 0.75,
   'dreb': 5.33,
   'reb': 6.08,
   'ast': 6.33,
   'stl': 1.67,
   'blk': 0.5,
   'turnover': 3.0,
   'pf': 1.33,
   'pts': 25.25,
   'fg_pct': 0.436,
   'fg3_pct': 0.396,
   'ft_pct': 0.947}],
 [],
 [{'games_played': 12,
   'player_id': 472,
   'season': 2021,
   'min': '34:12',
   'fgm': 6.92,
   'fga': 17.08,
   'fg3m': 1.0,
   'fg3a': 3.83,
   'ftm': 2.83,
   'fta': 4.42,
   'oreb': 1.58,
   'dreb': 7.0,
   'reb': 8.58,
   'ast': 8.17,
   'stl': 1.5,
   'blk': 0.17,
   'turnover': 4.92,
   'pf': 3.08,
   'pts': 17.67,
   'fg_pct': 0.405,
   'fg3_pct': 0.261,
   'ft_pct': 0.642}],
 [{'games_played': 12,
   'player_id': 192,
   'season': 2021,
   'min': '33:53',
   'fgm': 5.58,
   'fga': 13.67,
   'fg3m': 2.92,
   'fg3a': 7.42,
   'ftm': 4.08,
   'fta': 4.67,
   'oreb': 1.17,
   'dreb': 6.8

In [11]:
averages_df = pd.DataFrame(list(chain.from_iterable(averages_list)))
averages_df

,games_played,player_id,season,min,fgm,fga,fg3m,fg3a,ftm,fta,...,reb,ast,stl,blk,turnover,pf,pts,fg_pct,fg3_pct,ft_pct
0,12,115,2021,31:13,8.00,18.33,4.75,12.00,4.50,4.75,...,6.08,6.33,1.67,0.50,3.00,1.33,25.25,0.436,0.396,0.947
1,12,472,2021,34:12,6.92,17.08,1.00,3.83,2.83,4.42,...,8.58,8.17,1.50,0.17,4.92,3.08,17.67,0.405,0.261,0.642
2,12,192,2021,33:53,5.58,13.67,2.92,7.42,4.08,4.67,...,8.00,8.75,1.08,0.83,4.67,2.33,18.17,0.409,0.393,0.875
3,13,278,2021,33:34,7.31,18.62,2.62,9.31,2.69,3.15,...,4.15,7.69,0.62,0.08,2.46,1.62,19.92,0.393,0.281,0.854
4,6,237,2021,37:03,9.50,20.33,2.83,8.17,3.00,3.83,...,5.50,7.00,2.33,0.83,4.17,2.33,24.83,0.467,0.347,0.783
5,12,140,2021,34:27,11.17,19.08,1.92,4.75,5.25,6.33,...,8.42,5.00,0.58,0.67,3.50,1.33,29.50,0.585,0.404,0.829
6,12,15,2021,32:52,9.50,19.17,1.33,4.67,6.25,9.08,...,11.83,6.00,1.08,1.75,3.00,3.00,26.58,0.496,0.286,0.688
7,10,172,2021,35:01,9.90,21.60,3.20,8.80,3.70,4.30,...,8.30,5.40,2.50,0.50,4.60,3.00,26.70,0.458,0.364,0.860
8,11,79,2021,32:59,8.00,15.18,0.45,1.55,7.18,8.45,...,5.45,5.09,2.09,0.36,1.91,2.00,23.64,0.527,0.294,0.849
9,6,200,2021,33:03,8.50,15.67,1.33,4.17,1.50,1.67,...,9.00,4.17,0.17,0.33,1.50,1.17,19.83,0.543,0.320,0.900


In [12]:
%store -r salary_df
salary_df


,Rk,Player,Tm,2021-22
0,1,Stephen Curry,GSW,$45780966.00
1,2,John Wall,HOU,$44310840.00
2,3,Russell Westbrook,LAL,$44211146.00
3,4,James Harden,BRK,$43848000.00
4,5,Damian Lillard,POR,$43750000.00
5,6,LeBron James,LAL,$41180544.00
6,7,Kevin Durant,BRK,$40918900.00
7,8,Giannis Antetokounmpo,MIL,$39344970.00
8,9,Paul George,LAC,$39344970.00
9,10,Kawhi Leonard,LAC,$39344900.00


In [13]:
player_id_df = pd.DataFrame(player_id_list)
salary_id_join = player_id_df.join(salary_df)
salary_id_join

,0,Rk,Player,Tm,2021-22
0,115,1,Stephen Curry,GSW,$45780966.00
1,1014,2,John Wall,HOU,$44310840.00
2,472,3,Russell Westbrook,LAL,$44211146.00
3,192,4,James Harden,BRK,$43848000.00
4,278,5,Damian Lillard,POR,$43750000.00
5,237,6,LeBron James,LAL,$41180544.00
6,140,7,Kevin Durant,BRK,$40918900.00
7,15,8,Giannis Antetokounmpo,MIL,$39344970.00
8,172,9,Paul George,LAC,$39344970.00
9,274,10,Kawhi Leonard,LAC,$39344900.00


In [14]:
salary_player_id_df = salary_id_join.rename(columns={0: 'player_id'})
salary_player_id_df

,player_id,Rk,Player,Tm,2021-22
0,115,1,Stephen Curry,GSW,$45780966.00
1,1014,2,John Wall,HOU,$44310840.00
2,472,3,Russell Westbrook,LAL,$44211146.00
3,192,4,James Harden,BRK,$43848000.00
4,278,5,Damian Lillard,POR,$43750000.00
5,237,6,LeBron James,LAL,$41180544.00
6,140,7,Kevin Durant,BRK,$40918900.00
7,15,8,Giannis Antetokounmpo,MIL,$39344970.00
8,172,9,Paul George,LAC,$39344970.00
9,274,10,Kawhi Leonard,LAC,$39344900.00


In [15]:
all_data_df = salary_player_id_df.merge(averages_df, on='player_id', how = 'outer')
all_data_df.set_index('Rk').fillna(0)

,player_id,Player,Tm,2021-22,games_played,season,min,fgm,fga,fg3m,...,reb,ast,stl,blk,turnover,pf,pts,fg_pct,fg3_pct,ft_pct
Rk,,,,,,,,,,,,,,,,,,,,,
1,115,Stephen Curry,GSW,$45780966.00,12.0,2021.0,31:13,8.00,18.33,4.75,...,6.08,6.33,1.67,0.50,3.00,1.33,25.25,0.436,0.396,0.947
2,1014,John Wall,HOU,$44310840.00,0.0,0.0,0,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000
3,472,Russell Westbrook,LAL,$44211146.00,12.0,2021.0,34:12,6.92,17.08,1.00,...,8.58,8.17,1.50,0.17,4.92,3.08,17.67,0.405,0.261,0.642
4,192,James Harden,BRK,$43848000.00,12.0,2021.0,33:53,5.58,13.67,2.92,...,8.00,8.75,1.08,0.83,4.67,2.33,18.17,0.409,0.393,0.875
5,278,Damian Lillard,POR,$43750000.00,13.0,2021.0,33:34,7.31,18.62,2.62,...,4.15,7.69,0.62,0.08,2.46,1.62,19.92,0.393,0.281,0.854
6,237,LeBron James,LAL,$41180544.00,6.0,2021.0,37:03,9.50,20.33,2.83,...,5.50,7.00,2.33,0.83,4.17,2.33,24.83,0.467,0.347,0.783
7,140,Kevin Durant,BRK,$40918900.00,12.0,2021.0,34:27,11.17,19.08,1.92,...,8.42,5.00,0.58,0.67,3.50,1.33,29.50,0.585,0.404,0.829
8,15,Giannis Antetokounmpo,MIL,$39344970.00,12.0,2021.0,32:52,9.50,19.17,1.33,...,11.83,6.00,1.08,1.75,3.00,3.00,26.58,0.496,0.286,0.688
9,172,Paul George,LAC,$39344970.00,10.0,2021.0,35:01,9.90,21.60,3.20,...,8.30,5.40,2.50,0.50,4.60,3.00,26.70,0.458,0.364,0.860


In [16]:
# Game Score = Points Scored + (0.4 x Field Goals) – (0.7 x Field Goal Attempts) – (0.4 x (Free Throw Attempts – Free Throws)) + 
# (0.7 x Offensive Rebounds) + (0.3 x Defensive Rebounds) + Steals + (0.7 x Assists) + (0.7 x Blocks) – (0.4 x Personal Fouls) – Turnovers

In [48]:
steph = all_data_df.loc[0]
a = steph.loc['pts']
b = steph.loc['fga'] * 0.4
c = steph.loc['fga'] * 0.7
d = ((steph.loc['fta']) - (steph.loc['ftm']) * 0.4)
e = steph.loc['oreb'] * 0.7
f = steph.loc['dreb'] * 0.3
g = steph.loc['stl']
h = steph.loc['ast'] * 0.7
i = steph.loc['blk'] * 0.7
j = steph.loc['pf'] * 0.4
k = steph.loc['turnover']
game_score_steph = a + b - c - d + e + 
game_score_steph

In [41]:
steph


player_id                 115
Rk                          1
Player          Stephen Curry
Tm                        GSW
2021-22          $45780966.00
games_played             12.0
season                 2021.0
min                     31:13
fgm                       8.0
fga                     18.33
fg3m                     4.75
fg3a                     12.0
ftm                       4.5
fta                      4.75
oreb                     0.75
dreb                     5.33
reb                      6.08
ast                      6.33
stl                      1.67
blk                       0.5
turnover                  3.0
pf                       1.33
pts                     25.25
fg_pct                  0.436
fg3_pct                 0.396
ft_pct                  0.947
Name: 0, dtype: object